In [1]:
import json
import pandas as pd
import random

In [2]:
class Sentiment:
    Positive = 'POSITIVE'
    Negative = 'NEGATIVE'

class  Review:
    def __init__(self, text, score):
        self.text = text
        self.score = score
        self.sentiment =self.get_sentiment()


    def get_sentiment(self):
        if self.score <= 2:
            return Sentiment.Negative
        else:
            return Sentiment.Positive


class ReviewContainer:
    def __init__(self, reviews):
        self.reviews = reviews

    def get_text(self):
        return [x.text for x in self.reviews]

    def get_sentiment(self):
        return [x.sentiment for x in self.reviews]


    def evenly_distribute(self):
        negative = list(filter(lambda x: x.sentiment == Sentiment.Negative, self.reviews))
        positive = list(filter(lambda x: x.sentiment == Sentiment.Positive, self.reviews))
        positive_less = positive[:len(negative)]
        self.reviews = negative + positive_less
        random.shuffle(self.reviews)



In [3]:
file_name = "C:/Users/Hp Pc/Downloads/Books_small_10000.json"


reviews = []
with open(file_name) as f:
    for line in f:
        review = json.loads(line)
        reviews.append(Review(review['reviewText'], review['overall']))
        
    

In [4]:
reviews[0].text

"I bought both boxed sets, books 1-5.  Really a great series!  Start book 1 three weeks ago and just finished book 5.  Sloane Monroe is a great character and being able to follow her through both private life and her PI life gets a reader very involved!  Although clues may be right in front of the reader, there are twists and turns that keep one guessing until the last page!  These are books you won't be disappointed with."

PREP DATA

In [5]:
from sklearn.model_selection import train_test_split, RandomizedSearchCV

In [6]:
train, test = train_test_split(reviews, test_size=0.3, random_state=42)

train_cont = ReviewContainer(train)
test_cont = ReviewContainer(test)


In [7]:
train_cont

In [8]:
train_cont.evenly_distribute()
train_x = train_cont.get_text()
train_y = train_cont.get_sentiment()

test_cont.evenly_distribute()
test_x = test_cont.get_text()
test_y = test_cont.get_sentiment()

In [9]:
pd.value_counts(test_y)

C:\Users\Hp Pc\AppData\Local\Temp\ipykernel_4128\2061098744.py:1: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  pd.value_counts(test_y)
C:\Users\Hp Pc\AppData\Local\Temp\ipykernel_4128\2061098744.py:1: FutureWarning: value_counts with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  pd.value_counts(test_y)


POSITIVE    183
NEGATIVE    183
Name: count, dtype: int64

In [10]:
pd.value_counts(train_y)
#the positive and negative results are greatly not balance

C:\Users\Hp Pc\AppData\Local\Temp\ipykernel_4128\3166649834.py:1: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  pd.value_counts(train_y)
C:\Users\Hp Pc\AppData\Local\Temp\ipykernel_4128\3166649834.py:1: FutureWarning: value_counts with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  pd.value_counts(train_y)


POSITIVE    461
NEGATIVE    461
Name: count, dtype: int64

In [11]:
#Bag of Words Tokenizer

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer


In [12]:
vectorizer = TfidfVectorizer(binary=True, ngram_range=(1,2))

train_vector = vectorizer.fit_transform(train_x)
test_vector = vectorizer.transform(test_x)

CLASSIFICATION

In [13]:
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score

In [14]:
model_params = {
    'svm': {
        'model' : LinearSVC(random_state=42, dual=False),
        'params': {
           'penalty': ['l1', 'l2'],
            'loss': ['squared_hinge'],
            'C': [1.0, 3.0, 5.0]
        }
    },

    'rf_clf': {

        'model': RandomForestClassifier(random_state=42),
        'params': {
            "n_estimators" : [50, 100, 200, 300],
            "max_depth": [5,10,15,20],
            "max_leaf_nodes": [2,5,7],
            "min_samples_leaf": [2,4,7,10]
        }
    } 
}

In [15]:
scores = []

for model_name, mp in model_params.items():
    clf = RandomizedSearchCV(mp['model'], mp['params'], cv=5, random_state=42, n_iter=3, error_score='raise')
    clf.fit(train_vector.toarray(), train_y)
    scores.append({
        'model': model_name,
        'best_score': clf.best_score_,
        'best_params': clf.best_params_
    })

In [16]:
scores

[{'model': 'svm',
  'best_score': np.float64(0.8275381903642772),
  'best_params': {'penalty': 'l2', 'loss': 'squared_hinge', 'C': 1.0}},
 {'model': 'rf_clf',
  'best_score': np.float64(0.7201410105757933),
  'best_params': {'n_estimators': 50,
   'min_samples_leaf': 7,
   'max_leaf_nodes': 7,
   'max_depth': 15}}]

In [17]:
rf_clf = RandomForestClassifier(min_samples_leaf=10, max_leaf_nodes =7, max_depth=5, random_state=42 )

svc_clf = LinearSVC()

In [18]:
rf_clf.fit(train_vector, train_y)

RandomForestClassifier(max_depth=5, max_leaf_nodes=7, min_samples_leaf=10,
                       random_state=42)

In [19]:
svc_clf.fit(train_vector,train_y)

LinearSVC()

In [20]:
print(rf_clf.score(test_vector, test_y))     #RandomForest Accuracy on test sets

print(svc_clf.score(test_vector,test_y))

0.7076502732240437
0.8306010928961749


In [21]:
rf_f1 = f1_score(test_y, rf_clf.predict(test_vector), labels = [Sentiment.Positive, Sentiment.Negative], average=None)
svc_f1 = f1_score(test_y, svc_clf.predict(test_vector), average=None, labels = [Sentiment.Positive, Sentiment.Negative])


print(f"Random Forest f1_score:{rf_f1}")
print(f"Linear SVC f1_score: {svc_f1}")

Random Forest f1_score:[0.72207792 0.69164265]
Linear SVC f1_score: [0.82681564 0.8342246 ]


QUALITATIVE TEST

In [22]:
test_words = ["The books weren't so great", 'the novel was educational', 'the story of lagod taught me alot', "the novel wasn't so bad"]

word_vector = vectorizer.transform(test_words)

rf_clf.predict(word_vector)

array(['POSITIVE', 'POSITIVE', 'POSITIVE', 'POSITIVE'], dtype='<U8')

In [23]:
svc_clf.predict(word_vector)

#both model are not able to recognize abridged words = weren't and wasn't

array(['POSITIVE', 'POSITIVE', 'POSITIVE', 'NEGATIVE'], dtype='<U8')

In [24]:
test_words_2 = ['Do not buy this book', ' A total waste of time', 'a horrible story line']
word_2_vector = vectorizer.transform(test_words_2)

svc_clf.predict(word_2_vector)

array(['NEGATIVE', 'NEGATIVE', 'NEGATIVE'], dtype='<U8')

In [25]:
rf_clf.predict(word_2_vector)

array(['NEGATIVE', 'POSITIVE', 'POSITIVE'], dtype='<U8')

WORD VECTORS

using spaCy lib to add more meaning and understanding to the texts

import nltk

#nltk.download('popular')
#nltk.download('wordset')
#nltk.download('stopwords')
#nltk.download('punkt')

SAVING THE MODEL

In [34]:
import pickle

In [35]:
with open('C:/Users/Hp Pc/natural_language_processing/svc_model.pkl', 'wb') as f:
    pickle.dump(svc_clf, f)

In [36]:
with open('C:/Users/Hp Pc/natural_language_processing/svc_model.pkl', 'rb') as f:
    classifier = pickle.load(f, encoding='utf-8')

In [37]:
classifier

LinearSVC()